In [10]:
# =================================================================
# 1. SETUP & IMPORTS
# =================================================================
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from google.colab import drive

# Mount Drive and Set Path
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
DATA_PATH = '/content/drive/MyDrive/nfl-big-data-bowl-2022/'

In [12]:
# =================================================================
# 2. DATA LOADING & PREPROCESSING
# =================================================================
def calculate_kick_angle(absolute_yardline):
    dist_to_posts = abs(120 - absolute_yardline)
    return np.degrees(np.arctan(9.25 / (dist_to_posts * 3 + 1e-6)))

def load_data():
    print("Loading Big Data Bowl files...")
    plays = pd.read_csv(DATA_PATH + 'plays.csv')
    scouting = pd.read_csv(DATA_PATH + 'PFFScoutingData.csv')
    players = pd.read_csv(DATA_PATH + 'players.csv')
    players.columns = [c[0].lower() + c[1:] if c != 'nflId' else 'nflId' for c in players.columns]

    df = plays.merge(scouting, on=['gameId', 'playId'], how='left')
    df = df.merge(players[['nflId', 'displayName', 'position']], left_on='kickerId', right_on='nflId', how='left')

    # 1. Kickers Logic
    k_df = df[df['specialTeamsPlayType'].isin(['Field Goal', 'Extra Point'])].copy()
    k_df['is_good'] = (k_df['specialTeamsResult'] == 'Kick Attempt Good').astype(int)
    k_df['kick_angle'] = k_df['absoluteYardlineNumber'].apply(calculate_kick_angle)

    X_k_log = k_df[['kickLength']].fillna(k_df['kickLength'].median())
    y_k_log = k_df['is_good']
    log_reg = LogisticRegression().fit(X_k_log, y_k_log)
    k_df['expected_prob'] = log_reg.predict_proba(X_k_log)[:, 1]
    k_df['poe'] = k_df['is_good'] - k_df['expected_prob']

    # 2. Punters Logic
    p_df = df[df['specialTeamsPlayType'] == 'Punt'].copy()
    X_p = p_df[['absoluteYardlineNumber']].fillna(60)
    y_p = p_df['playResult'].fillna(0)
    p_baseline = RandomForestRegressor(n_estimators=50).fit(X_p, y_p)
    p_df['expected_net'] = p_baseline.predict(X_p)
    p_df['fpoe'] = p_df['playResult'] - p_df['expected_net']

    return k_df, p_df, players

df_kicks, df_punts, players_meta = load_data()

Loading Big Data Bowl files...


In [13]:
# =================================================================
# 3. BAYESIAN AXIAL TRANSFORMER + MATERN KERNEL SMOOTHING ENGINE
# =================================================================
class MaternKernelAttention(nn.Module): # smoothing engine
    def __init__(self, d_model):
        super().__init__()
        self.lengthscale = nn.Parameter(torch.tensor([1.0]))
    def forward(self, x, time_indices):
        dist_matrix = torch.abs(time_indices.unsqueeze(-1) - time_indices.unsqueeze(1)) #calc time distance between each kick
        kernel_weights = torch.exp(-dist_matrix / (self.lengthscale + 1e-6)) #implementing matern 1/2 kernel so that kicker skill is seen as a continuous flow
        return torch.matmul(kernel_weights, x) # meant to smooth the noise in a kicker's history

class BayesianAxialTransformer(nn.Module):
    def __init__(self, in_features, d_model=64):
        super().__init__()
        self.embed = nn.Linear(in_features, d_model)
        self.col_attn = nn.MultiheadAttention(d_model, 4, batch_first=True) # helps look sideways across features
        self.gp_row_attn = MaternKernelAttention(d_model)
        self.norm = nn.LayerNorm(d_model)
        self.skill_head = nn.Linear(d_model, 2) # outputs two variables skill and uncertainty
    def forward(self, x, time_indices, difficulty):
        x = self.embed(x)
        res = x
        x_feat, _ = self.col_attn(x, x, x)
        x = self.norm(x_feat + res)
        x = self.gp_row_attn(x, time_indices)
        skill_params = self.skill_head(x)
        mu, log_var = skill_params[..., 0], skill_params[..., 1]
        prob = torch.sigmoid(mu - (difficulty / 100.0)) # informs the model that the success is determinded by skill and diffuculty
        return prob, mu, torch.exp(log_var)

In [14]:
# =====================
# 4. TENSOR PREPARATION / Converting the kicking players for each player into a 3D tensor of [Kickers, Plays, Features]
# =====================
def prepare_tensors(df, feature_cols, max_plays=50):
    kicker_groups = df.groupby('kickerId') # separating by kicker id
    X_l, t_l, d_l, target_l = [], [], [], []
    for _, group in kicker_groups:
        feats = group[feature_cols].fillna(0).values
        times = np.arange(len(group))
        dists = group['kickLength'].fillna(0).values if 'kickLength' in group else group['absoluteYardlineNumber'].values
        targs = group['is_good'].values if 'is_good' in group else (group['fpoe'] > 0).astype(int).values
        if len(feats) > max_plays:
            feats, times, dists, targs = feats[:max_plays], times[:max_plays], dists[:max_plays], targs[:max_plays]
        else:
            p = max_plays - len(feats)
            feats = np.vstack([feats, np.zeros((p, len(feature_cols)))])
            times, dists, targs = np.append(times, np.zeros(p)), np.append(dists, np.zeros(p)), np.append(targs, np.zeros(p))
        X_l.append(feats); t_l.append(times); d_l.append(dists); target_l.append(targs)
    return torch.FloatTensor(np.array(X_l)), torch.FloatTensor(np.array(t_l)), torch.FloatTensor(np.array(d_l)), torch.FloatTensor(np.array(target_l))

def train_bayesian_model(model, X, t, d, targets, epochs=50):
    optimizer = optim.AdamW(model.parameters(), lr=0.001) # using adamW algorithm to nudge the weights into the right direction
    for epoch in range(epochs):
        optimizer.zero_grad()
        probs, mu, var = model(X, t, d)
        perf_loss = F.binary_cross_entropy(probs, targets) #punishes the model for fuessing incorrectly
        smooth_loss = torch.mean((mu[:, 1:] - mu[:, :-1])**2)
        loss = perf_loss + 0.5 * smooth_loss
        loss.backward()
        optimizer.step()
    return model

In [15]:
# =================================================================
# 5. INITIALIZE, TRAIN, AND ELO UPDATES
# =================================================================
# Prepare Tensors
print("Preparing tensors...")
k_X, k_t, k_d, k_target = prepare_tensors(df_kicks, ['kickLength', 'kick_angle', 'operationTime', 'poe'])
p_X, p_t, p_d, p_target = prepare_tensors(df_punts, ['kickLength', 'hangTime', 'operationTime', 'fpoe'])

# Train Models
print("Training models...")
k_model = train_bayesian_model(BayesianAxialTransformer(4), k_X, k_t, k_d, k_target)
p_model = train_bayesian_model(BayesianAxialTransformer(4), p_X, p_t, p_d, p_target)

# Elo Update Logic
def update_elo(current_elo, actual_score, expected_prob, k_factor=32):
    return current_elo + k_factor * (actual_score - expected_prob)

def calculate_leaderboard(df, model, X, t, d, meta):
    model.eval()
    with torch.no_grad():
        probs, mu, var = model(X, t, d)
        m, s = mu[:, -1].numpy(), torch.sqrt(var[:, -1]).numpy()
        probs_np = probs.numpy()

    # Calculate Bayesian Rank Score
    k_ids = df.groupby('kickerId')['displayName'].first().reset_index()
    k_ids['Rank_Score'] = m - (3 * s)

    # Calculate Elo
    elo_ratings = []
    for i in range(len(k_ids)):
        current_elo = 1500.0
        for j in range(len(probs_np[i])):
            if j >= len(df[df['kickerId'] == k_ids.iloc[i]['kickerId']]): break
            actual = X[i, j, -1].item() > 0 # Use POE/FPOE > 0 as a win proxy
            current_elo = update_elo(current_elo, actual, probs_np[i, j])
        elo_ratings.append(current_elo)

    k_ids['Elo_Rating'] = elo_ratings
    return k_ids.sort_values('Rank_Score', ascending=False)

Preparing tensors...
Training models...


In [19]:
import torch
import numpy as np

# 1. TENSOR PREPARATION LOGIC
def prepare_belo_tensors(df, feature_cols, max_plays=50):
    kicker_groups = df.groupby('kickerId')
    X_l, t_l, d_l, target_l = [], [], [], []
    for _, group in kicker_groups:
        feats = group[feature_cols].fillna(0).values
        times = np.arange(len(group))
        # Distance used for the f_theta penalty in the model
        dists = group['kickLength'].fillna(0).values
        targs = group['is_good'].values

        # Ensure uniform 3D shape [Kickers, Plays, Features]
        if len(feats) > max_plays:
            feats, times, dists, targs = feats[:max_plays], times[:max_plays], dists[:max_plays], targs[:max_plays]
        else:
            p = max_plays - len(feats)
            feats = np.vstack([feats, np.zeros((p, len(feature_cols)))])
            times, dists, targs = np.append(times, np.zeros(p)), np.append(dists, np.zeros(p)), np.append(targs, np.zeros(p))

        X_l.append(feats); t_l.append(times); d_l.append(dists); target_l.append(targs)

    return (torch.FloatTensor(np.array(X_l)), torch.FloatTensor(np.array(t_l)),
            torch.FloatTensor(np.array(d_l)), torch.FloatTensor(np.array(target_l)))

# Generate the tensors
feature_list = ['kickLength', 'kick_angle', 'operationTime', 'poe']
X_tensor, time_indices, dist_tensor, target_tensor = prepare_belo_tensors(df_kicks, feature_list)

# 2. INITIALIZE AND TRAIN MODEL
# We initialize the model with 4 features to match feature_list
model = BayesianAxialTransformer(in_features=4)

print("Training Bayesian Axial Transformer...")
# Run for 50 epochs to establish the Bayesian prior
model = train_bayesian_model(model, X_tensor, time_indices, dist_tensor, target_tensor, epochs=50)

# 3. CALCULATE FILTERED LEADERBOARD
# We use the 'minimum kicks' filter we discussed (50% of league average)
filtered_leaderboard = calculate_belo_with_filter(df_kicks, model, X_tensor, time_indices, dist_tensor)

print("\n--- FINAL QUALIFIED KICKER POWER RANKINGS ---")
print(filtered_leaderboard[['displayName', 'attempt_count', 'BELO_Score', 'Power_Rank']].head(1000))

Training Bayesian Axial Transformer...
League Avg Attempts: 102.0
Qualifying Threshold (>50 kicks):

--- FINAL QUALIFIED KICKER POWER RANKINGS ---
             displayName  attempt_count   BELO_Score  Power_Rank
33              Wil Lutz            232  2068.907897  100.000000
44       Michael Badgley            145  2042.023133   94.249725
36         Aldrick Rosas            126  2026.937634   91.023148
25           Jason Myers            197  2014.640019   88.392861
3            Matt Bryant             84  1991.141383   83.366831
19         Justin Tucker            231  1980.784881   81.151721
32            Josh Lambo            106  1971.115984   79.083679
6     Stephen Gostkowski            165  1968.881402   78.605733
40       Harrison Butker            241  1960.480586   76.808916
22       Brandon McManus            166  1950.287112   74.628675
5           Robbie Gould            173  1938.867973   72.186281
15         Randy Bullock            160  1929.885324   70.265018
21      

In [22]:
def calculate_belo_with_filter(df, model, X, t, d, threshold_ratio=0.5):
    # 1. Calculate filtering threshold
    attempts_per_player = df.groupby('kickerId').size()
    avg_attempts = attempts_per_player.mean()
    min_attempts = int(avg_attempts * threshold_ratio) # creating a filtering threshold based on the league average for min attempts

    print(f"League Avg Attempts: {avg_attempts:.1f} | Qualifying Threshold: {min_attempts}")

    model.eval()
    with torch.no_grad():
        probs, _, var = model(X, t, d)
        probs, sigmas = probs.cpu().numpy(), torch.sqrt(var).cpu().numpy()

    # Meta-data mapping
    p_meta = df.groupby('kickerId')['displayName'].first().reset_index()
    p_meta['attempt_count'] = p_meta['kickerId'].map(attempts_per_player).values

    # 2. Identify the Success Column
    # If 'is_good' isn't there, we use (fpoe > 0) as the 'Actual' success
    success_col = 'is_good' if 'is_good' in df.columns else 'fpoe_success'
    if success_col == 'fpoe_success' and 'fpoe_success' not in df.columns:
        df['fpoe_success'] = (df['fpoe'] > 0).astype(int)

    belo_scores = []
    for i in range(len(p_meta)):
        current_belo = 1500.0
        p_id = p_meta.iloc[i]['kickerId']
        # Sort by game/play to ensure chronological Elo updates
        p_plays = df[df['kickerId'] == p_id].sort_values(['gameId', 'playId'])

        for j in range(min(len(p_plays), 50)):
            actual = p_plays.iloc[j][success_col]
            expected = probs[i, j]
            uncertainty = sigmas[i, j]

            # Update Elo: Score += K * (1 + sigma) * (Actual - Expected)
            current_belo += 32 * (1 + uncertainty) * (actual - expected)
        belo_scores.append(current_belo)

    p_meta['BELO_Score'] = belo_scores

    # 3. Filter and Normalize
    qualified_lb = p_meta[p_meta['attempt_count'] >= min_attempts].copy()
    if not qualified_lb.empty:
        min_b, max_b = qualified_lb['BELO_Score'].min(), qualified_lb['BELO_Score'].max()
        qualified_lb['Power_Rank'] = 100 * (qualified_lb['BELO_Score'] - min_b) / (max_b - min_b + 1e-6)

    return qualified_lb.sort_values('Power_Rank', ascending=False)

# Now run it for punters
p_leaderboard = calculate_belo_with_filter(df_punts, p_model, p_X_tensor, p_time_indices, p_dist_tensor)

print("\n--- FINAL QUALIFIED PUNTER POWER RANKINGS (FPOE-BELO) ---")
print(p_leaderboard[['displayName', 'attempt_count', 'BELO_Score', 'Power_Rank']].head(1000))

League Avg Attempts: 111.6 | Qualifying Threshold: 55

--- FINAL QUALIFIED PUNTER POWER RANKINGS (FPOE-BELO) ---
          displayName  attempt_count   BELO_Score  Power_Rank
24          Matt Wile             69  1968.438963  100.000000
46           Jack Fox             56  1918.888614   94.760136
6     Thomas Morstead            156  1884.280126   91.100349
11      Johnny Hekker            175  1872.684176   89.874097
31  Rigoberto Sanchez            156  1852.094683   87.696794
45       Jamie Gillan            104  1821.884369   84.502106
3            Sam Koch            152  1815.366610   83.812864
35    Michael Dickson            195  1799.247251   82.108270
4          Brett Kern            184  1761.724232   78.140276
43        Jake Bailey            125  1754.861021   77.414503
32         Matt Haack            214  1727.157512   74.484905
15          Tress Way            219  1679.682358   69.464490
33   Cameron Johnston            188  1634.591826   64.696244
8         Matt Bosh

In [36]:
import torch
import numpy as np
import torch.nn.functional as F

# --- 1. THE MISSING FUNCTION DEFINITION ---
def calculate_belo_with_filter(df, model, X, t, d, threshold_ratio=0.5):
    # Calculate filtering threshold (50% of league average)
    attempts_per_player = df.groupby('kickerId').size()
    avg_attempts = attempts_per_player.mean()
    min_attempts = int(avg_attempts * threshold_ratio)

    print(f"League Avg Attempts: {avg_attempts:.1f} | Qualifying Threshold: {min_attempts}")

    model.eval()
    with torch.no_grad():
        # Get probabilities and uncertainty (sigma) from the Transformer
        probs, _, var = model(X, t, d)
        probs, sigmas = probs.cpu().numpy(), torch.sqrt(var).cpu().numpy()

    # Meta-data mapping for names
    k_meta = df.groupby('kickerId')['displayName'].first().reset_index()
    k_meta['attempt_count'] = k_meta['kickerId'].map(attempts_per_player).values

    # Identify success metric
    success_col = 'is_good' if 'is_good' in df.columns else 'fpoe_success'
    if success_col == 'fpoe_success' and 'fpoe_success' not in df.columns:
        df['fpoe_success'] = (df['fpoe'] > 0).astype(int)

    belo_scores = []
    for i in range(len(k_meta)):
        current_belo = 1500.0 # Standard Elo starting point
        k_id = k_meta.iloc[i]['kickerId']
        k_plays = df[df['kickerId'] == k_id].sort_values(['gameId', 'playId'])

        for j in range(min(len(k_plays), 50)):
            actual = k_plays.iloc[j][success_col]
            expected = probs[i, j]
            uncertainty = sigmas[i, j]

            # Math: Score += 32 * (1 + Uncertainty) * (Actual - Expected)
            current_belo += 32 * (1 + uncertainty) * (actual - expected)
        belo_scores.append(current_belo)

    k_meta['BELO_Score'] = belo_scores

    # Apply Minimum Kicks Filter
    qualified_lb = k_meta[k_meta['attempt_count'] >= min_attempts].copy()

    # Normalize Power Rank (0-100)
    if not qualified_lb.empty:
        min_b, max_b = qualified_lb['BELO_Score'].min(), qualified_lb['BELO_Score'].max()
        qualified_lb['Power_Rank'] = 100 * (qualified_lb['BELO_Score'] - min_b) / (max_b - min_b + 1e-6)

    return qualified_lb.sort_values('Power_Rank', ascending=False)

# --- 2. TENSOR PREPARATION ---
feature_list = ['kickLength', 'kick_angle', 'operationTime', 'poe']
X_tensor, time_indices, dist_tensor, target_tensor = prepare_belo_tensors(df_kicks, feature_list)

# --- 3. MODEL INITIALIZATION & TRAINING ---
model = BayesianAxialTransformer(in_features=4)
print("Training Bayesian Axial Transformer...")
model = train_bayesian_model(model, X_tensor, time_indices, dist_tensor, target_tensor, epochs=50)

# --- 4. EXECUTE RANKING ---
filtered_leaderboard = calculate_belo_with_filter(df_kicks, model, X_tensor, time_indices, dist_tensor)

print("\n--- FINAL QUALIFIED KICKER POWER RANKINGS ---")
print(filtered_leaderboard[['displayName', 'attempt_count', 'BELO_Score', 'Power_Rank']].head(2000))

Training Bayesian Axial Transformer...
League Avg Attempts: 102.0 | Qualifying Threshold: 50

--- FINAL QUALIFIED KICKER POWER RANKINGS ---
             displayName  attempt_count   BELO_Score  Power_Rank
36         Aldrick Rosas            126  1845.369798  100.000000
33              Wil Lutz            232  1842.580978   99.442298
25           Jason Myers            197  1833.400345   97.606376
5           Robbie Gould            173  1779.560829   86.839673
32            Josh Lambo            106  1778.955058   86.718532
19         Justin Tucker            231  1770.340273   84.995767
44       Michael Badgley            145  1769.406759   84.809085
3            Matt Bryant             84  1754.489713   81.826009
43         Jason Sanders            173  1744.512697   79.830828
6     Stephen Gostkowski            165  1739.445793   78.817560
7            Matt Prater            182  1712.146161   73.358242
40       Harrison Butker            241  1704.749937   71.879162
22       Brando

In [35]:
# 1. ADD SUCCESS PROXY TO PUNTER DATA
# We define a "win" for a punter as anytime they exceed the expected net yardage
df_punts['fpoe_success'] = (df_punts['fpoe'] > 0).astype(int)

# Define a specialized tensor preparation function for punters
def prepare_belo_tensors_punter(df, feature_cols, max_plays=50):
    punter_groups = df.groupby('kickerId') # In this dataset, 'kickerId' is used for punters too
    X_l, t_l, d_l, target_l = [], [], [], []
    for _, group in punter_groups:
        feats = group[feature_cols].fillna(0).values
        times = np.arange(len(group))
        # For punters, 'difficulty' might be related to field position
        dists = group['absoluteYardlineNumber'].fillna(0).values # Use absoluteYardlineNumber for punter difficulty
        targs = group['fpoe_success'].values # Use fpoe_success as the target for punters

        # Ensure uniform 3D shape [Punters, Plays, Features]
        if len(feats) > max_plays:
            feats, times, dists, targs = feats[:max_plays], times[:max_plays], dists[:max_plays], targs[:max_plays]
        else:
            p = max_plays - len(feats)
            feats = np.vstack([feats, np.zeros((p, len(feature_cols)))])
            times, dists, targs = np.append(times, np.zeros(p)), np.append(dists, np.zeros(p)), np.append(targs, np.zeros(p))
        X_l.append(feats); t_l.append(times); d_l.append(dists); target_l.append(targs)
    return (torch.FloatTensor(np.array(X_l)), torch.FloatTensor(np.array(t_l)),
            torch.FloatTensor(np.array(d_l)), torch.FloatTensor(np.array(target_l)))


# 2. PREPARE PUNTER TENSORS
p_features = ['kickLength', 'hangTime', 'operationTime', 'fpoe']
p_X, p_t, p_d, p_target = prepare_belo_tensors_punter(df_punts, p_features) # Use the new punter-specific function

# 3. INITIALIZE AND TRAIN PUNTER MODEL
p_model = BayesianAxialTransformer(in_features=4)
print("Training Bayesian Axial Transformer for Punters...")
p_model = train_bayesian_model(p_model, p_X, p_t, p_d, p_target, epochs=50)

# 4. RUN RANKING WITH THE FLEXIBLE FUNCTION
# This function now checks for 'is_good' or 'fpoe_success'
def calculate_belo_with_filter_v2(df, model, X, t, d, threshold_ratio=0.5):
    attempts_per_player = df.groupby('kickerId').size()
    avg_attempts = attempts_per_player.mean()
    min_attempts = int(avg_attempts * threshold_ratio)

    model.eval()
    with torch.no_grad():
        probs, _, var = model(X, t, d)
        probs, sigmas = probs.cpu().numpy(), torch.sqrt(var).cpu().numpy()

    p_meta = df.groupby('kickerId')['displayName'].first().reset_index()
    p_meta['attempt_count'] = p_meta['kickerId'].map(attempts_per_player).values

    # Logic to switch between Kicker and Punter success metrics
    success_col = 'is_good' if 'is_good' in df.columns else 'fpoe_success'

    belo_scores = []
    for i in range(len(p_meta)):
        current_belo = 1500.0
        p_id = p_meta.iloc[i]['kickerId']
        p_plays = df[df['kickerId'] == p_id].sort_values(['gameId', 'playId'])

        for j in range(min(len(p_plays), 50)):
            actual = p_plays.iloc[j][success_col]
            expected = probs[i, j]
            uncertainty = sigmas[i, j]
            current_belo += 32 * (1 + uncertainty) * (actual - expected)
        belo_scores.append(current_belo)

    p_meta['BELO_Score'] = belo_scores
    qualified_lb = p_meta[p_meta['attempt_count'] >= min_attempts].copy()

    if not qualified_lb.empty:
        min_b, max_b = qualified_lb['BELO_Score'].min(), qualified_lb['BELO_Score'].max()
        qualified_lb['Power_Rank'] = 100 * (qualified_lb['BELO_Score'] - min_b) / (max_b - min_b + 1e-6)

    return qualified_lb.sort_values('Power_Rank', ascending=False)

# Execute the Punter Ranking
punter_leaderboard = calculate_belo_with_filter_v2(df_punts, p_model, p_X, p_t, p_d)

print("\n--- FINAL QUALIFIED PUNTER POWER RANKINGS (FPOE-BELO) ---")
print(punter_leaderboard[['displayName', 'attempt_count', 'BELO_Score', 'Power_Rank']].head(2000))

Training Bayesian Axial Transformer for Punters...

--- FINAL QUALIFIED PUNTER POWER RANKINGS (FPOE-BELO) ---
          displayName  attempt_count   BELO_Score  Power_Rank
46           Jack Fox             56  2040.193573  100.000000
6     Thomas Morstead            156  2009.339634   96.845775
4          Brett Kern            184  1946.402854   90.411695
43        Jake Bailey            125  1911.939201   86.888446
15          Tress Way            219  1880.716515   83.696524
31  Rigoberto Sanchez            156  1820.786503   77.569828
32         Matt Haack            214  1793.694044   74.800142
24          Matt Wile             69  1731.111708   68.402297
45       Jamie Gillan            104  1716.058289   66.863373
11      Johnny Hekker            175  1712.966458   66.547292
26        Riley Dixon            200  1703.140794   65.542806
42   Mitch Wishnowsky            109  1697.049864   64.920125
2     Dustin Colquitt            104  1690.603292   64.261087
35    Michael Dickson 